# 

<Details>
   <Summary>1. Abstract</Summary>

Kidney function prediction for individuals consider living kidney donation

ABSTRACT     
**Background**: Individuals who donate a kidney lose 50% of their nephron mass. Kidney reserve capacity varies by individual characteristics. Reduced 6-month post-donation creatinine-based estimated glomerular filtration rate (eGFRcr) is associated with increased long-term risk of kidney failure. We sought to inform the individual predicted 6-month post-donation creatinine and expected eGFRcr using the race-free CKD-EPI 2021.   
   
**Methods**: We examined the U.S. national cohort of living kidney donors between 01/2005-06/2019. We used donor demographic and health characteristics to develop the prediction model, which was validated by 10-fold cross-validation and temporal validation. We assessed bias, precision, and accuracy of model performance between race groups.    
   
**Results**: Of 52494 donors included, median (IQR) predonation and 6-month post-donation creatinine and eGFRcr were 0.8 mg/dL (0.7, 0.9) and 101 mL/min/1.73m² (88, 112), and 1.2 mg/dL (1.0, 1.4) and 65 mL/min/1.73m² (56, 75), respectively. Our model had predictive R²=0.68 which was consistent in cross-validation (R²=0.68) and temporal validation (R²=0.71). The model retained correct classification of expected 6-month eGFRcr <50 or ≥50 mL/min/1.73m² in 90% of donors. The median (IQR) bias of expected 6-month eGFRcr was 2.02 mL/min/1.73m² (-3.86, 7.08) among Black donors and -0.82 mL/min/1.73m² (-7.06, 4.72) among non-Black donors. The expected 6-month eGFRcr was within 30% of observed eGFRcr among 95% of Black donors and among 97% non-Black donors.    
    
**Conclusions**: We provide an accurate prediction of 6-month post-donation kidney function and a calculator for precision counseling of individuals considering kidney donation. Our tool helps improve decision-making for marginal donor candidates and identify those who need careful surveillance.

`alammary_LivingKidneyDonor_post_donation_6month_eGFR_prediction.docx`

</Details>

<Details>
   <Summary>2. Regression</Summary>

```stata

      name:  <unnamed>
       log:  C:\Users\Fawaz\OneDrive - The University of Colorado Denver\Creator 1\FawazResearch\CODE LKD\living_donor_2023\\cr_6m_predict_20230901.log
  log type:  text
 opened on:   1 Sep 2023, 09:45:14

. 
. ***Study of LKD donor 6-M eGFR prediction*********
. 
. 
. use "${DTA}cr6m_predict", clear 
(DONOR_LIVE                      )

. 
. /*
> 1SCr per 1 mg/dL increase 
> 2Age per 1 year increase
> ³BMI per 1 kg/m2 increase
> ⁴Height per 1 meter increase 
> */
. 
. regress dfl_ki_creat ///
>         don_ki_creat_preop cr_sp07 cr_sp09 ///
>         age age_sp55_10 ///
>     male ///
>         c.don_ki_creat_preop#male c.cr_sp07#male ///
>         nbmi bmi_sp30 ///
>         ht_100 ///
>         htn  

      Source |       SS           df       MS      Number of obs   =    52,494
-------------+----------------------------------   F(12, 52481)    =   9260.13
       Model |  2227.76986        12  185.647489   Prob > F        =    0.0000
    Residual |  1052.14192    52,481  .020048054   R-squared       =    0.6792
-------------+----------------------------------   Adj R-squared   =    0.6791
       Total |  3279.91178    52,493  .062482841   Root MSE        =    .14159

-------------------------------------------------------------------------------------------
             dfl_ki_creat | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
--------------------------+----------------------------------------------------------------
       don_ki_creat_preop |   .8191583   .0156349    52.39   0.000     .7885138    .8498029
                  cr_sp07 |   .1311153   .0221774     5.91   0.000     .0876474    .1745832
                  cr_sp09 |  -.1581432   .0213985    -7.39   0.000    -.2000844    -.116202
                      age |   .0024177   .0000664    36.40   0.000     .0022875    .0025479
              age_sp55_10 |  -.0071846   .0028369    -2.53   0.011     -.012745   -.0016242
                     male |   .3429115   .0577508     5.94   0.000     .2297194    .4561036
                          |
male#c.don_ki_creat_preop |
                       1  |  -.3593172   .0837424    -4.29   0.000    -.5234532   -.1951812
                          |
           male#c.cr_sp07 |
                       1  |   .4733182   .0890624     5.31   0.000     .2987551    .6478812
                          |
                     nbmi |   .0034174   .0002168    15.76   0.000     .0029924    .0038424
                 bmi_sp30 |  -.0025009   .0006129    -4.08   0.000    -.0037023   -.0012995
                   ht_100 |   .1290259    .008752    14.74   0.000     .1118718    .1461799
                      htn |   .0074556   .0032935     2.26   0.024     .0010003     .013911
                    _cons |   .0600343   .0181453     3.31   0.001     .0244693    .0955994
-------------------------------------------------------------------------------------------

. 
. predict yhat
(option xb assumed; fitted values)

. 
. 
. gen yhat_egfr_21  = 142 * ///
>         min((yhat/cond(male, 0.9, 0.7)), 1)^cond(male, -0.302, -0.241)  * ///
>         max((yhat/cond(male, 0.9, 0.7)), 1)^(-1.2) * ///        
>         0.9938^age * cond(male, 1, 1.012) // add follow up age

. 
.         sum don_ki_creat_preop bfd_ckdepi2021 dfl_ki_creat ad_ckdepi2021 yhat yhat_egfr_21

    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
don_ki_cre~p |     52,494    .8271839    .1708142         .3        1.5
bfd_ckd~2021 |     52,477     100.037    15.80717   50.13269   164.1908
dfl_ki_creat |     52,494    1.207251    .2499657        .53        2.4
ad_ckde~2021 |     52,494    66.41345    14.58557   24.96214   133.7426
        yhat |     52,494    1.207251    .2060082    .655553   1.954492
-------------+---------------------------------------------------------
yhat_egfr_21 |     52,494    65.48574    11.13556   33.34264    128.189

. 
.         format %9.2f don_ki_creat_preop yhat dfl_ki_creat

.         format %9.0f bfd_ckdepi2021 yhat_egfr_21 ad_ckdepi2021

.                 format %9.0f nbmi

.                         format %9.2f ht_100

. list age male race_cat nbmi ht_100 htn sbp smoke don_ki_creat_preop bfd_ckdepi2021 yhat yhat_egfr_21 dfl_ki_creat ad_ckdepi2021 in 1/5, table   

     +-------------------------------------------------------------------------------------------------------------------------+
     | age   male   race_cat   nbmi   ht_100   htn   sbp   smoke   don_ki~p   bfd~2021   yhat   yhat_~21   dfl_ki~t   ad_~2021 |
     |-------------------------------------------------------------------------------------------------------------------------|
  1. |  56      1      black     27     1.83     0   122       1       1.10         79   1.58         51       1.80         44 |
  2. |  63      0      white     22     1.68     0   120       0       0.90         72   1.26         48       1.40         42 |
  3. |  27      0      black     20     1.65     0   116       0       1.30         58   1.49         49       1.60         45 |
  4. |  46      0      black     33     1.61     0   132       0       0.98         72   1.31         51       1.45         45 |
  5. |  63      0      white     29     1.55     0   118       0       0.80         83   1.18         52       1.26         48 |
     +-------------------------------------------------------------------------------------------------------------------------+

. 
. log close
      name:  <unnamed>
       log:  C:\Users\Fawaz\OneDrive - The University of Colorado Denver\Creator 1\FawazResearch\CODE LKD\living_donor_2023\\cr_6m_predict_20230901.log
  log type:  text
 closed on:   1 Sep 2023, 09:45:16

```

</Details>

<Details>
   <Summary>3. Exemplars</Summary>
Links to nice examples of online calculators 
http://www.transplantmodels.com/ (this is a website developed by collaborators at Johns Hopkins and then they moved to NYU).
http://www.transplantmodels.com/lkdpi/
http://www.transplantmodels.com/esrdrisk/
https://ckdpcrisk.org/pcr2acr_adj/
 
Thus, we only need to develop a calculator as the senior author of this article works at NYU and he will require to deploy the calculator at their website. So I think here is the best example:
http://www.transplantmodels.com/esrdrisk/
The outcome will be:
Predicted post-donation six-month kidney function.
Predicted Creatinine            Expected eGFRcr*
 
*CKD-EPI creatinine equation (2021)

</Details>

<Details>
   <Summary>4. Script</Summary>

```stata
//Fawaz needs a calculator
//Project Prediction of 6-m post-donation Cr and calcluate expected eGFR(2021) based on predicted creatinine

capture log close
clear all
macro drop all
set more off
set linesize 200
timer clear  

local cdat: di %tdCCYYNNDD date(c(current_date),"DMY") 
global cdat: di %tdCCYYNNDD date(c(current_date),"DMY") 
quietly di "`cdat'" 


global FILE = "C:\Users\Fawaz\OneDrive - The University of Colorado Denver\Creator 1\FawazResearch\CODE LKD\living_donor_2023\"

global DATA = "C:\Users\Fawaz\OneDrive - The University of Colorado Denver\Creator 1\FawazResearch\CODE LKD\living_donor_2023\dta"

global FIG "${FILE}\fig\"
global OUT "${FILE}\output\"
global DTA "${FILE}\dta\"

log using "${FILE}\cr_6m_predict_`cdat'.log", replace 

***Study of LKD donor 6-M eGFR prediction*********


use "${DTA}cr6m_predict", clear 

/*
1SCr per 1 mg/dL increase 
2Age per 1 year increase
³BMI per 1 kg/m2 increase
⁴Height per 1 meter increase 
*/

regress dfl_ki_creat ///
	don_ki_creat_preop cr_sp07 cr_sp09 ///
	age age_sp55_10 ///
    male ///
	c.don_ki_creat_preop#male c.cr_sp07#male ///
	nbmi bmi_sp30 ///
	ht_100 ///
	htn  

predict yhat


gen yhat_egfr_21  = 142 * ///
	min((yhat/cond(male, 0.9, 0.7)), 1)^cond(male, -0.302, -0.241)  * ///
	max((yhat/cond(male, 0.9, 0.7)), 1)^(-1.2) * ///	
	0.9938^age * cond(male, 1, 1.012)

	sum don_ki_creat_preop bfd_ckdepi2021 dfl_ki_creat ad_ckdepi2021 yhat yhat_egfr_21

	format %9.2f don_ki_creat_preop yhat dfl_ki_creat
	format %9.0f bfd_ckdepi2021 yhat_egfr_21 ad_ckdepi2021
		format %9.0f nbmi
			format %9.2f ht_100
list age male race_cat nbmi ht_100 htn sbp smoke don_ki_creat_preop bfd_ckdepi2021 yhat yhat_egfr_21 dfl_ki_creat ad_ckdepi2021 in 1/5, table	

log close

```
</Details>

<Details>
   <Summary>5. Formula</Summary>
Post-donation 6-m creatinine prediction equation 
Equation: 6-month post-donation serum creatinine = 0.06 + [0.8192 (– 0.3593 if male)]*(predonation creatinine) + [0.1311 (+ 0.4733 if male)]*(predonation creatinine – 0.7)*(predonation creatinine>0.7) – 0.1581*(predonation creatinine – 0.9)*(predonation creatinine>0.9) + 0.3429*(male) + 0.0034*(BMI) – 0.0025*(BMI – 30)*(BMI>30) + 0.0024*(age) – 0.0007*(age – 55)*(age>55) + 0.1290*(height in meters) + 0.0075*(hypertension)
 
SCr per 1 mg/dL increase  2Age per 1 year increase
BMI per 1 kg/m2 increase
Height per 1 meter increase 
 
predict yhat
Calculate the expected 6-m eGFR (CKDepi 2021) using the predicted creatinine from the above 
gen yhat_egfr_21  = 142 * ///
            min((yhat/cond(male, 0.9, 0.7)), 1)^cond(male, -0.302, -0.241)  * ///
            max((yhat/cond(male, 0.9, 0.7)), 1)^(-1.2) * ///  
            0.9938^age * cond(male, 1, 1.012) 
 
Format number to be easily read for providers
            format %9.2f don_ki_creat_preop yhat dfl_ki_creat
            format %9.0f bfd_ckdepi2021 yhat_egfr_21 ad_ckdepi2021
                        format %9.0f nbmi
                                    format %9.2f ht_100

</Details>

